<img src="https://github.com/krairy/utilities/blob/main/Linser%20Lab.JPG?raw=true" height="50" align="left" style="height:60px"> 

<img src="https://bmrb.io/images/BMRB_logo_icon.svg" height="100" align="left" style="height:60px">

# BRMB vis
This is a tool to visualise assignments from [BMRB](https://bmrb.io/).

Simply run all cells below and follow the instructions!

In [ ]:
#@title Install dependencies
%%bash

pip install -q --no-warn-conflicts pynmrstar

In [ ]:
#@title Import libraries and fetching the data

import pynmrstar
import pandas as pd
import matplotlib.pyplot as plt
import sys
from io import StringIO

#@markdown Enter the BMRB ID 
#@markdown For example, 51228 for Alex's SH3 assignments

try:
    bmrb_id = 51228 #@param {type:"integer"} 
    nmrstar = pynmrstar.Entry.from_database(bmrb_id)

except ValueError:
    print("Not a number")
except OSError:
    print(f"Entry {bmrb_id} does not exist in the public database")
    
nmrstar = pynmrstar.Entry.from_database(bmrb_id)
loops = nmrstar.get_saveframes_by_category('assigned_chemical_shifts')[0]
shifts = loops['_Atom_chem_shift']

## print(shifts)

data = StringIO(shifts.get_data_as_csv().replace("_Atom_chem_shift.", ""))
df = pd.read_csv(data, index_col=0)

## print(df.columns)

#@markdown Raw datafrom NMR Star file

df.tail(10)

In [ ]:
#@title Rearranging the data

df1 = df["Seq_ID Comp_ID Atom_ID Val Val_err Ambiguity_code".split()].set_index("Seq_ID Comp_ID Atom_ID".split())
df1.columns = "ChemShift CS_error Ambiguity".split()
df1

In [ ]:
#@title Visualisation
#@markdown Select chemical shifts for X and Y axes:

dimX = "H" #@param "H N C CA CB HA".split() {type:"string"}
dimY = "N" #@param "H N C CA CB HA".split() {type:"string"}

x = df1["ChemShift"].loc[:, :, dimX]
y = df1["ChemShift"].loc[:, :, dimY]

df_2D = pd.merge(x, y, on="Seq_ID", suffixes = (f'_{dimX}', f'_{dimY}'))
fig = df_2D.plot(x=f"ChemShift_{dimX}", y=f"ChemShift_{dimY}", kind="scatter", 
                 backend="plotly", text=df_2D.index,
          )

fig.update_xaxes(
    autorange="reversed")
fig.update_yaxes(
    autorange="reversed")
fig.update_traces(textposition='top left')